In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn import metrics
import json
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
import collections
from itertools import groupby
import pickle
import re
import matplotlib.colors as mcolors
import warnings
import requests
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from functools import reduce
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

### Global Variables & Functions

#### Variables

In [3]:
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

with open('../../data/supportive/colour_list.json', 'r') as f:
  colors = json.load(f)

with open('../../data/supportive/trait_dictionary.json', 'r') as f:
  traits_dict = json.load(f)
traits_list = list(traits_dict.keys())
traits_list += [trait.lower() for lst in list(traits_dict.values()) for trait in lst]

#### Functions

In [4]:
def corresponding_keys(val, dictionary):
    """returns the corresponding key of a single value 
    assuming the values are lists.

    Args:
        val (string): string present in the dict
        dictionary (dict): dict with lists of stings as values

    Returns:
        list: list of matching keys
    """
    # Init list
    keys = []
    # Search the dict
    for k, v in dictionary.items():
        if val in v:
            keys.append(k)
    return keys
    
def jaccard_similarity(A, B):
    """Calculates the Jaccard similarity two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    A = set(A.lower().split())
    B = set(B.lower().split())
    
    # Get intersection of two sets
    nominator = A.intersection(B)

    # Find union of two sets
    denominator = A.union(B)

    # Take the ratio of sizes
    similarity = len(nominator)/len(denominator)

    return similarity/len(A)


def similarity(groundtruth, pred):
    """Calculates the normal similarity between two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    groundtruth = set(groundtruth)
    pred = set(pred)

    # Find intersection of two sets
    nominator = groundtruth.intersection(pred)

    # Find union of two sets
    denominator = groundtruth

    # Take the ratio of sizes
    similarity = len(nominator)/(len(denominator))

    return similarity

def resentesize(lst):

    sentence = ''

    # Loop every 2 items 
    for item1, item2 in zip(lst[::2], lst[1::2]):
        
        # Set verb
        verb = 'has'
        chunk = item2.split(' ')[-1]
        if chunk not in nouns or chunk in colors:
            verb = 'is'

        sentence += f'{item1.capitalize()} {verb} {item2}. '
    
    return sentence


def get_wiki_main_image(title):
    url = 'https://en.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : title
    }
    response = requests.get(url, data)
    json_data = json.loads(response.text)
    return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'


### Load GT Data

In [5]:
root = "../../data/external/"

#### Data Andrei

In [6]:
file_name = root + 'Dataset_Andrei.csv'
df_Andrei = pd.read_csv(file_name)

# Get Dummies to match DF Pierre
df_Andrei_dummies = pd.get_dummies(df_Andrei.iloc[:, 2:])
# Set species back
df_Andrei_dummies = df_Andrei_dummies.set_index(df_Andrei['Species'])

# Create tuple list for multi index
Andrei_multi_index = []
for top_index in df_Andrei.columns:
    for sub_index in df_Andrei_dummies.columns:
        if top_index in sub_index:

            sub_index = sub_index.split('_')[-1]
            Andrei_multi_index.append((top_index, sub_index))

# Set Mutli index
df_Andrei_dummies.columns = pd.MultiIndex.from_tuples(Andrei_multi_index)
df_Andrei_dummies

Life form      Leaf position                      \
                               liana tree     alternate alternate, opposite   
Species                                                                       
Avicennia germinans                0    1             0                   0   
Metopium brownei                   0    1             1                   0   
Handroanthus billbergii            0    1             0                   0   
Bourreria succulenta               0    1             1                   0   
Bursera karsteniana                0    1             1                   0   
Bursera simaruba                   0    1             1                   0   
Bursera tomentosa                  0    1             1                   0   
Cynophalla flexuosa                1    0             1                   0   
Cynophalla hastata                 0    1             1                   0   
Quadrella indica                   0    1             1                   0   
Quadrella odoratissima             0    1             1                   0   
Crossopetalum rhacoma              0    1             0                   0   
Maytenus tetragona                 0    1             1                   0   
Maytenus versluysii                0    1             1                   0   
Clusia rosea                       0    1             0                   0   
Conocarpus erectus                 0    1             1                   0   
Laguncularia racemosa              0    1             0                   0   
Erythroxylum havanense             0    1             1                   0   
Croton niveus                      0    1             1                   0   
Hippomane mancinella               0    1             1                   0   
Libidibia coriaria                 0    1             1                   0   
Pithecellobium unguis-cati         0    1             1                   0   
Prosopis juliflora                 0    1             1                   0   
Vachellia tortuosa                 0    1             1                   0   
Vitex compressa                    0    1             0                   0   
Vitex cymosa                       0    1             0                   0   
Trichilia trifolia                 0    1             1                   0   
Eugenia procera                    0    1             0                   0   
Guapira pacurero                   0    1             0                   0   
Coccoloba swartzii                 0    1             1                   0   
Coccoloba uvifera                  0    1             1                   0   
Krugiodendron ferreum              0    1             0                   1   
Rhizophora mangle                  0    1             0                   0   
Guettarda roupalifolia             0    1             0                   0   
Randia aculeata                    0    1             0                   0   
Amyris ignea                       0    1             1                   0   
Zanthoxylum monophyllum            0    1             1                   0   
Casearia tremula                   0    1             1                   0   
Schoepfia schreberi                0    1             1                   0   
Jacquinia arborea                  0    1             1                   0   
Guaiacum officinale                0    1             0                   0   
Guaiacum sanctum                   0    1             0                   0   

                                                           \
                           opposite opposite, whorls of 3   
Species                                                     
Avicennia germinans               1                     0   
Metopium brownei                  0                     0   
Handroanthus billbergii           1                     0   
Bourreria succulenta              0                     0   
Bursera karsteniana               0         

#### Data Pierre

In [7]:
file_name = root + 'Dataset_Pierre.csv'
df_Pierre = pd.read_csv(file_name, header=[0, 1]) 
df_Pierre = df_Pierre.iloc[: , 1:]
df_Pierre = df_Pierre.set_index(df_Pierre['Species']['species'])
df_Pierre = df_Pierre.drop(columns=['Species', 'XXX'])

/var/folders/0p/7b4jnnp15d5gx16h36l4ycx00000gn/T/ipykernel_41551/4153551112.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_Pierre = df_Pierre.drop(columns=['Species', 'XXX'])


#### Data Palms

In [8]:
file_name = root + 'Dataset_Kissling.txt'
df_Daniel = pd.read_csv(file_name,
                 sep='\t', encoding='Latin-1')
palm_species = df_Daniel[~df_Daniel.isnull().any(axis=1)]['SpecName'].values
df_Daniel.set_index('SpecName', inplace=True)

### RESHAPE DATA FOR PALMS

# Exclude string types
df_Daniels_int = df_Daniel.select_dtypes(exclude=[object])
df_Daniels_str = df_Daniel.select_dtypes(include=[object])
# Drop numbers
df_Daniels_semi_ints = df_Daniels_int.loc[:, df_Daniels_int.max() <= 3]
# Merge again
df_Daniel_edit = pd.merge(df_Daniels_str, df_Daniels_semi_ints, left_index=True, right_index=True)


# Real numbers:
df_Daniels_real_ints = df_Daniels_int.loc[:, df_Daniels_int.max() >= 3]
columns = [
    ("Measurement", "Maximum Stem Height in Meters"),
    ("Measurement", "Maximum Stem Diameter in Centimeters"),
    ("Measurement", "Maximum Leaf Number"),
    ("Measurement", "Maximum Leaf Blade Length in Meters"),
    ("Measurement", "Maximum Rachis Length in Meters"),
    ("Measurement", "Maximum Petiole Length in Meters"),
    ("Measurement", "Average Fruit Length in Centimeters"),
    ("Measurement", "Minimum Fruit Length in Centimeters"),
    ("Measurement", "Maximum Fruit Length in Centimeters"),
    ("Measurement", "Average Fruit Width in Centimeters"),
    ("Measurement", "Minimum Fruit Width in Centimeters"),
    ("Measurement", "Maximum Fruit Width in Centimeters"),
]

df_Daniels_real_ints.columns = pd.MultiIndex.from_tuples(columns)
df_Daniels_real_ints

# Get colors as lst of lsts 
FruitColorDescription_colors_lst = []

for palm_colors in df_Daniels_str['FruitColorDescription'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        FruitColorDescription_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        FruitColorDescription_colors_lst.append([])

MainFruitColors_colors_lst = []

for palm_colors in df_Daniels_str['MainFruitColors'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        MainFruitColors_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        MainFruitColors_colors_lst.append([])

# Init SKlearn MLB
mlb = MultiLabelBinarizer()

# Create dummies for color columns
df_FruitColorDescription = pd.DataFrame(
    {
        'FruitColorDescription': FruitColorDescription_colors_lst
    }, columns=['FruitColorDescription'])

s = df_FruitColorDescription['FruitColorDescription']
df_FruitColorDescription = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('Fruit Colour Description', column) for column in df_FruitColorDescription.columns]
df_FruitColorDescription.columns = pd.MultiIndex.from_tuples(columns)

# Create dummies for color columns
df_MainFruitColors = pd.DataFrame(
    {
        'MainFruitColors': MainFruitColors_colors_lst
    }, columns=['MainFruitColors'])

s = df_MainFruitColors['MainFruitColors']
df_MainFruitColors = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('Fruit Colour', column) for column in df_MainFruitColors.columns]
df_MainFruitColors.columns = pd.MultiIndex.from_tuples(columns)

df_Daniels_str_non_color = df_Daniels_str[['UnderstoreyCanopy', 'FruitSizeCategorical', 'FruitShape', 'Conspicuousness']]

# df_Daniels_str_non_color.columns = pd.MultiIndex.from_tuples(
#     [
#         ('Crown', 'UnderstoreyCanopy'),
#         ('Fruit Size', 'FruitSizeCategorical'),
#         ('Fruit Shape', 'FruitShape'),
#         ('Conspicuousness', 'Conspicuousness'),
#     ]
# )

df_Daniels_str_non_color_dummies = pd.get_dummies(df_Daniels_str_non_color)
columns = []
for column in df_Daniels_str_non_color_dummies.columns:
    level0, level1 = column.split('_')
    if level0 == 'UnderstoreyCanopy':
        level0 = 'Crown'
    elif level0 == 'FruitSizeCategorical':
        level0 = 'Fruit Size'
    elif level0 == 'FruitShape':
        level0 = 'Fruit Shape'
    elif level0 == 'Conspicuousness':
        level0 = 'Conspicuousness'
    columns.append((level0, level1))
    
df_Daniels_str_non_color_dummies.columns = pd.MultiIndex.from_tuples(columns)

### JOIN ALL DATA
data_frames = [df_FruitColorDescription, df_MainFruitColors, df_Daniels_str_non_color_dummies, df_Daniels_real_ints]
df_Daniel_merged = pd.concat(data_frames, axis=1)
df_Daniel_merged

Fruit Colour Description                             \
                                            black blue bluish brown brownish   
SpecName                                                                       
Acanthophoenix crinita                          1    0      0     0        0   
Acanthophoenix rousselii                        1    0      0     0        0   
Acanthophoenix rubra                            1    0      0     0        0   
Acoelorrhaphe wrightii                          1    0      0     1        0   
Acrocomia aculeata                              0    0      0     0        0   
...                                           ...  ...    ...   ...      ...   
Wettinia quinaria                               0    0      0     1        0   
Wettinia radiata                                0    0      0     0        0   
Wettinia verruculosa                            0    0      0     1        0   
Wodyetia bifurcata                              0    0      0     0        0   
Zombia antillarum                               0    0      0     0        0   

                                                                  ...  \
                         chocolate coral crimson darkgreen green  ...   
SpecName                                                          ...   
Acanthophoenix crinita           0     0       0         0     0  ...   
Acanthophoenix rousselii         0     0       0         0     0  ...   
Acanthophoenix rubra             0     0       0         0     0  ...   
Acoelorrhaphe wrightii           0     0       0         0     0  ...   
Acrocomia aculeata               0     0       0         0     1  ...   
...                            ...   ...     ...       ...   ...  ...   
Wettinia quinaria                0     0       0         0     1  ...   
Wettinia radiata                 0     0       0         0     0  ...   
Wettinia verruculosa             0     0       0         0     1  ...   
Wodyetia bifurcata               0     0       0         0     0  ...   
Zombia antillarum                0     0       0         0     0  ...   

                                 Measurement  \
                         Maximum Leaf Number   
SpecName                                       
Acanthophoenix crinita                  15.0   
Acanthophoenix rousselii                 NaN   
Acanthophoenix rubra                    20.0   
Acoelorrhaphe wrightii                  25.0   
Acrocomia aculeata                      30.0   
...                                      ...   
Wettinia quinaria                        6.0   
Wettinia radiata                         6.0   
Wettinia verruculosa                     6.0   
Wodyetia bifurcata                      10.0   
Zombia antillarum                       12.0   

                                                              \
                         Maximum Leaf Blade Length in Meters   
SpecName                                                       
Acanthophoenix crinita                                  2.30   
Acanthophoenix rousselii                                3.00   
Acanthophoenix rubra                                    3.10   
Acoelorrhaphe wrightii                                  1.30   
Acrocomia aculeata                                      3.50   
...                                                      ...   
Wettinia quinaria                                       5.50   
Wettinia radiata                                        4.33   
Wettinia verruculosa                                    6.00   
Wodyetia bifurcata                                      3.20   
Zombia antillarum                                       1.20   

                                                          \
                         Maximum Rachis Length in Meters   
SpecName                                                   
Acanthophoenix crinita                               NaN   
Acanthophoenix rousselii                             NaN   
Acanthophoe

## Prediction Data

### Load Data

In [9]:
root = "../../data/processed/"
sentences_all = {}

f = open(F"{root}Sentences_Pierre.pkl", 'rb')
sentences_Pierre = pickle.load(f)
sentences_all |= sentences_Pierre 

f = open(F"{root}Sentences_Andrei.pkl", 'rb')
sentences_Andrei = pickle.load(f)
sentences_all |= sentences_Andrei 

f = open(F"{root}Sentences_Kissling.pkl", 'rb')
sentences_Kissling = pickle.load(f)
sentences_all |= sentences_Kissling 

# Drop duplicates
for species, sentences in sentences_all.items():
    sentences_all[species] = list(set(sentences))


### Match Data

In [30]:
k= 5
google_form_lst = []

for idx, (species, sentences) in enumerate(tqdm(sentences_all.items())):

    # if idx > 1:
    #     continue
    # if species != 'Dypsis thiryana':
    #     continue
    
    df_select = [df_Andrei_dummies, df_Pierre, df_Daniel_merged]
    if species in df_Andrei_dummies.index:
        df_select = df_select[0]
    elif species in df_Pierre.index:
        df_select = df_select[1]
    elif species in df_Daniel_merged.index:
        df_select = df_select[2]
    else:
        raise ValueError

    for gt_main_trait in df_select.columns.get_level_values(0).unique():
        if gt_main_trait == 'Measurement':
            df_subset = df_select[df_select.index == species][gt_main_trait]
            # Not really efficient, use PD?
    
            subtraits = list(df_subset.columns)
            values = df_subset.values[0]
            for sub_trait, value in zip(subtraits, values):

                df_sent  = F"{sub_trait}: {value}"

                top_list = []

                for sentence in sentences:
                    gt_sim = jaccard_similarity(df_sent, sentence)
                    if not gt_sim:
                        sentence = np.NaN
                    top_list.append((gt_sim, sentence))

                top_list.sort(reverse=True)
                # top_k_list = [sentence for (_, sentence) in top_list[0:k]]
                gt_sim_sum = 0
                top_k_list = []
                for (gt_sim, sentence) in top_list[0:k]:
                    gt_sim_sum += gt_sim
                    top_k_list.append(sentence)

                # Non Nan for normalization
                None_NaNs = k - top_k_list.count(np.NaN) + 2e-26 # Float division               

                google_form_lst.append((species, gt_main_trait, sub_trait, [df_sent], *top_k_list, df_sent.capitalize(), gt_sim_sum/None_NaNs))
        else:
            df_subset = df_select[df_select.index == species][gt_main_trait]
            present_traits = df_subset.loc[:, df_subset.any()].columns.values
            
            # df_sent  = ' '.join(gt_main_trait + ' ' + present_traits)
            
            # NEW VERSION SINGLE TRAIT
            size = present_traits.shape
            if not size[0]:
                continue
            df_sent = F"{gt_main_trait} {present_traits[0]}"
            # print(df_sent)

            top_list = []

            for sentence in sentences:
                gt_sim = jaccard_similarity(df_sent, sentence)

                if not gt_sim:
                    sentence = np.NaN

                # print(gt_sim, sentence, df_sent)
                top_list.append((gt_sim, sentence))

            top_list.sort(reverse=True)

            # print(top_list)

            # top_k_list = [sentence for (_, sentence) in top_list[0:k]]
            gt_sim_sum = 0
            top_k_list = []
            for (gt_sim, sentence) in top_list[0:k]:
                gt_sim_sum += gt_sim
                top_k_list.append(sentence)

            # Non Nan for normalization
            None_NaNs = k - top_k_list.count(np.NaN) + 2e-26 # Float division       

            # GoogleSent = gt_main_trait + ': ' + ', '.join(list(present_traits))
            GoogleSent = F"{gt_main_trait}: {present_traits[0]}"
            google_form_lst.append((species, gt_main_trait, gt_main_trait, list(present_traits), *top_k_list, GoogleSent.capitalize(), gt_sim_sum/None_NaNs))

100%|██████████| 647/647 [00:14<00:00, 44.30it/s]


### Create Subset

In [33]:
# Google forms
df_google = pd.DataFrame(google_form_lst, columns=['Species', 'Main Trait', 'GT Main Trait', 'GT Sub Traits', '1', '2', '3', '4', '5', 'GoogleSentence', 'Jaccard Sum'])
df_google

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Jaccard Sum
0,Acacia amythethophylla,plant type,plant type,"[Tree, Shrub]","A tree 20â40 ft. high, sometimes scarcely ar...","Small or medium tree 2â12 m. high, bark roug...","Small or medium tree 2-15 m. high, bark rough,...",Shrub or tree to 7 m. Thorns in pairs at nodes...,NaN,Plant type: tree,0.025783
1,Acacia amythethophylla,phyllotaxis,phyllotaxis,[Phyllotaxis alternate],NaN,NaN,NaN,NaN,NaN,Phyllotaxis: phyllotaxis alternate,0.000000
2,Acacia amythethophylla,trunk and root,trunk and root,[Base of trunk straight],"Inflorescence large, terminal, much-branched, ...",Acacia confusa - Root Smaller peaks suspected ...,"Pinnae sometimes 20 or more pairs, with 50 or ...","Flowers orange or yellow, strongly and sweetly...","Flowers orange or yellow, strongly and sweetly...",Trunk and root: base of trunk straight,0.010767
3,Acacia amythethophylla,latex,latex,[No exudate],NaN,NaN,NaN,NaN,NaN,Latex: no exudate,0.000000
4,Acacia amythethophylla,crown,crown,[Horizontally spreading crowns],NaN,NaN,NaN,NaN,NaN,Crown: horizontally spreading crowns,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
13141,Wodyetia bifurcata,Measurement,Minimum Fruit Length in Centimeters,[Minimum Fruit Length in Centimeters: 5.0],"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Fruit showing remains of style.,Minimum fruit length in centimeters: 5.0,0.017583
13142,Wodyetia bifurcata,Measurement,Maximum Fruit Length in Centimeters,[Maximum Fruit Length in Centimeters: 6.5],"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Fruit showing remains of style.,Maximum fruit length in centimeters: 6.5,0.017583
13143,Wodyetia bifurcata,Measurement,Average Fruit Width in Centimeters,[Average Fruit Width in Centimeters: 4.35],The average width of the crown of leaves on a ...,"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Average fruit width in centimeters: 4.35,0.019378
13144,Wodyetia bifurcata,Measurement,Minimum Fruit Width in Centimeters,[Minimum Fruit Width in Centimeters: 2.7],"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Fruit showing remains of style.,Minimum fruit width in centimeters: 2.7,0.017583


In [36]:
df_google_thresh = df_google[df_google["Jaccard Sum"] > 0.01]
df_google_thresh

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Jaccard Sum
0,Acacia amythethophylla,plant type,plant type,"[Tree, Shrub]","A tree 20â40 ft. high, sometimes scarcely ar...","Small or medium tree 2â12 m. high, bark roug...","Small or medium tree 2-15 m. high, bark rough,...",Shrub or tree to 7 m. Thorns in pairs at nodes...,NaN,Plant type: tree,0.025783
2,Acacia amythethophylla,trunk and root,trunk and root,[Base of trunk straight],"Inflorescence large, terminal, much-branched, ...",Acacia confusa - Root Smaller peaks suspected ...,"Pinnae sometimes 20 or more pairs, with 50 or ...","Flowers orange or yellow, strongly and sweetly...","Flowers orange or yellow, strongly and sweetly...",Trunk and root: base of trunk straight,0.010767
5,Acacia amythethophylla,stem shape,stem shape,[Circular stem section],Acacia confusa - Stem Smaller peaks suspected ...,NaN,NaN,NaN,NaN,Stem shape: circular stem section,0.020833
6,Acacia amythethophylla,bark,bark,"[Presence of spines or prickles on the trunk, ...",The branch bark is apparently 1.5% approx.,"Fruits flat, reddish-brown, remaining on the t...","Small or medium tree 2â12 m. high, bark roug...","Small or medium tree 2-15 m. high, bark rough,...","Pinnae sometimes 20 or more pairs, with 50 or ...",Bark: presence of spines or prickles on the trunk,0.013757
7,Acacia amythethophylla,bark color,bark color,"[Slice through pink bark, Slice through red ba...",The branch bark is apparently 1.5% approx.,"Small or medium tree 2â12 m. high, bark roug...","Small or medium tree 2-15 m. high, bark rough,...",NaN,NaN,Bark color: slice through pink bark,0.014949
...,...,...,...,...,...,...,...,...,...,...,...
13141,Wodyetia bifurcata,Measurement,Minimum Fruit Length in Centimeters,[Minimum Fruit Length in Centimeters: 5.0],"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Fruit showing remains of style.,Minimum fruit length in centimeters: 5.0,0.017583
13142,Wodyetia bifurcata,Measurement,Maximum Fruit Length in Centimeters,[Maximum Fruit Length in Centimeters: 6.5],"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Fruit showing remains of style.,Maximum fruit length in centimeters: 6.5,0.017583
13143,Wodyetia bifurcata,Measurement,Average Fruit Width in Centimeters,[Average Fruit Width in Centimeters: 4.35],The average width of the crown of leaves on a ...,"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Average fruit width in centimeters: 4.35,0.019378
13144,Wodyetia bifurcata,Measurement,Minimum Fruit Width in Centimeters,[Minimum Fruit Width in Centimeters: 2.7],"White, Pale-yellow in Summer.",Dark-green in All seasons.,"Fruit is 2.inches long, deep orange to deep re...",Mature red fruit on Wodyetia.,Fruit showing remains of style.,Minimum fruit width in centimeters: 2.7,0.017583


In [37]:
for i in range(10):
    df_google_subset = df_google_thresh\
                                     .sample(n=20, axis=0, random_state=333+i)

    df_google_subset.to_csv(f'{root}top_sents_all_AllSentencesAgainstTrait_Random20Subset_{i+1}.csv', sep='\t')



In [38]:
df_google_subset = df_google[df_google["Jaccard Sum"] > 0.02].sample(n=20, axis=0, random_state=123)
df_google_subset.to_csv(f'{root}top_sents_all_AllSentencesAgainstTrait_Random20Subset_demo.csv', sep='\t')
df_google_subset

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Jaccard Sum
7720,Zanthoxylum zanthoxyloides,fruit,fruit,[Capsule],"Fruit an ovoid follicle, 5â6 mm in diameter,...","When dried, the flesh of the fruit hardens, tu...",The fruit is usually of up to four follicles f...,General importance Geographic coverage Africa ...,NaN,Fruit: capsule,0.023172
4038,Guiera senegalensis,leaf shape,leaf shape,[Simple leaf],"They have simple , broad leaves and achene.",The against-constant leaves are short-stalked ...,NaN,NaN,NaN,Leaf shape: simple leaf,0.024603
9552,Chamaedorea costaricana,Crown,Crown,[canopy],"This is a tall, solitary species with a thick ...",NaN,NaN,NaN,NaN,Crown: canopy,0.020833
4150,Harungana madagascariensis,leaf shape,leaf shape,[Simple leaf],"Its leaves are opposite, simple and ovate.",Leaf blade underside is covered with stellate ...,At the tenth leaf stage: 'oil' glands appear t...,NaN,NaN,Leaf shape: simple leaf,0.025724
5051,Maerua angolensis,bark,bark,"[Smooth bark, Finely fissured bark]",The tree has a rounded crown and smooth grey b...,The bark on young stems turns from light purpl...,NaN,NaN,NaN,Bark: smooth bark,0.045833
5018,Lophira lanceolata,inflorescences,inflorescences,[Panicle of flowers or inflorescences],The inflorescence take the form of a loose pan...,Lophira lanceolata is a small deciduous tree g...,"The flowers of L. alata are white, fairly larg...",The leaves of the latter exude latex when dama...,The leaves are clustered at the end of the twigs.,Inflorescences: panicle of flowers or inflores...,0.020825
5542,Opilia celtidifolia,fruit,fruit,"[Drupe, or drupe-like fruit]","Fruit a drupe, ellipsoid, puberulous to shortl...",The fruit are 3 cm long.,Ripe fruit up to 3 cm.,"Flowers small, greenish-white or yellowish, fr...","Fruit up to 2 cm, ellipsoid, yellow to pale or...","Fruit: drupe, or drupe-like fruit",0.027516
4875,Leucaena leucocephala,leaf margin,leaf margin,[Smooth leaf margin],Green Leaf On White Background.,Leaves bipinnate with 4-9 pairs of pinnae per ...,These clusters are borne in the leaf forks on ...,A small raised structure is usually present on...,A small raised structure is usually present on...,Leaf margin: smooth leaf margin,0.022796
12341,Rhopaloblaste ceramica,Conspicuousness,Conspicuousness,[conspicuous],"Seed 21 - 31 mm long, 14 - 16 mm diam., ellips...","Seed 21 - 31 mm long, 14 - 16 mm in diam., ell...",NaN,NaN,NaN,Conspicuousness: conspicuous,0.025658
5849,Phyllanthus welwitschianus,bark,bark,"[Smooth bark, Finely fissured bark]","Leaves, roots, stem, bark and berries of this ...",NaN,NaN,NaN,NaN,Bark: smooth bark,0.035714
